<a href="https://colab.research.google.com/github/nccorwin/MSBA-6460-Final-Project/blob/main/6460_FinBERT_FB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers
# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast, TFAutoModel

# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import re
import numpy as np
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 3.2 MB/s 
     |████████████████████████████████| 101 kB 2.0 MB/s 
     |████████████████████████████████| 6.6 MB 36.9 MB/s 
     |████████████████████████████████| 596 kB 40.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
df = pd.read_csv('/content/drive/MyDrive/FB_posts_labeled.txt', delimiter="\t")

In [6]:
df.head()
# text = list(data['message'])

,postId,message,Appreciation,Complaint,Feedback
0,126016648090_10150802142013091,Great ! ;),1,0,0
1,108381603303_10151136215833304,YUM! YUM!,1,0,0
2,108381603303_3913438087739,Yummm :)),1,0,0
3,110455108974424_343049739048292,sweet,1,0,0
4,110455108974424_350358541650745,nice,1,0,0


In [7]:
# Clean_text found on kaggle notebook
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [8]:
df['message'] = df['message'].map(lambda x : clean_text(x))

In [9]:
train_sentences = df["message"].fillna("CVxTz").values
list_classes = ["Appreciation", "Complaint", "Feedback"]
train_y = df[list_classes].values

In [10]:
df.head()

,postId,message,Appreciation,Complaint,Feedback
0,126016648090_10150802142013091,great,1,0,0
1,108381603303_10151136215833304,yum yum,1,0,0
2,108381603303_3913438087739,yummm,1,0,0
3,110455108974424_343049739048292,sweet,1,0,0
4,110455108974424_350358541650745,nice,1,0,0


In [15]:
# Name of the BERT model to use
model_name = 'yiyanghkust/finbert-tone'

# Max length of tokens
max_length = 128

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name,num_labels=3)
#config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
finbert = TFAutoModel.from_pretrained(model_name)

Some layers from the model checkpoint at yiyanghkust/finbert-tone were not used when initializing TFBertModel: ['dropout_37', 'classifier']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at yiyanghkust/finbert-tone.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [16]:
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32') 
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
x = finbert.bert(inputs)

In [17]:
x

TFBaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                                 <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert')>),
                                                ('pooler_output',
                                                 <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'bert')>)])

In [14]:
#x2 =Dense(512, activation='relu')(x[1])
x2 = GlobalAveragePooling1D()(x[0])
#x3 = Dropout(0.4)(x2)
#x4 = Dense(256, activation='relu')(x3)
#x5 = Dropout(0.5)(x4)
y = Dense(len(list_classes), activation='softmax', name='outputs')(x2)

model = Model(inputs=inputs, outputs=y)
#model.layers[2].trainable = False

# Take a look at the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109751808   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [18]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [19]:
optimizer = Adam(lr=1e-5, decay=1e-6)
model.compile(loss = keras.losses.CategoricalCrossentropy(),
                  optimizer=optimizer,
                  metrics=['accuracy',f1_m])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
x = tokenizer(
    text=list(train_sentences),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [21]:
history = model.fit(
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    #x={'input_ids': x['input_ids']},
    y={'outputs': train_y},
    validation_split=0.2,
    batch_size=32,
    epochs=1)

199/199 [==============================] - 9271s 46s/step - loss: 0.6005 - accuracy: 0.7508 - f1_m: 0.7430 - val_loss: 0.4183 - val_accuracy: 0.8305 - val_f1_m: 0.8256


In [22]:
test_df = pd.read_csv('/content/drive/MyDrive/FB_posts_unlabeled.txt', delimiter='\t')
test_df['message']=test_df['message'].map(lambda x : clean_text(x))
test_sentences = test_df["message"].fillna("CVxTz").values

In [23]:
test_x = tokenizer(
    text=list(test_sentences),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [24]:
del test_sentences
del x
del df

In [25]:
import gc
gc.collect()

1563

In [26]:
test_x

{'input_ids': <tf.Tensor: shape=(2039, 128), dtype=int32, numpy=
array([[    3,  6635,    41, ...,     0,     0,     0],
       [    3,  3076,     4, ...,     0,     0,     0],
       [    3, 13511,     4, ...,     0,     0,     0],
       ...,
       [    3,   392,     4, ...,     0,     0,     0],
       [    3, 10409, 16854, ...,     0,     0,     0],
       [    3,    26,    17, ...,  1772,     9,     4]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2039, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>}

In [27]:
predictions=model.predict(x={'input_ids': test_x['input_ids'], 'attention_mask': test_x['attention_mask']},batch_size=32)

In [28]:
list_classes_test = ["Appreciation_pred", "Complaint_pred", "Feedback_pred"]

In [29]:
submission=pd.DataFrame(predictions,columns=list_classes_test)
submission['postId'] = test_df['postId']
submission=submission[['postId']+(list_classes_test)]
submission.to_csv("/content/drive/MyDrive/finalproject_FinBERTbaseuncased.csv", index=False)

In [30]:
submission=pd.DataFrame(predictions,columns=list_classes_test)
m = np.zeros_like(submission.values)
m[np.arange(len(submission)), submission.values.argmax(1)] = 1

df1 = pd.DataFrame(m, columns = submission.columns, index=submission.index).astype(int)
df1['postId'] = test_df['postId']
df1=df1[['postId']+(list_classes_test)]
print(df1)
df1.to_csv("/content/drive/MyDrive/finalproject_FinBERTbaseuncased_rounded.csv", index=False)

                               postId  Appreciation_pred  Complaint_pred  \
0      108381603303_10151119973393304                  1               0   
1     115568331790246_371841206162956                  1               0   
2     115568331790246_515044031842672                  1               0   
3      147285781446_10151010892176447                  1               0   
4      159616034235_10150639103634236                  1               0   
...                               ...                ...             ...   
2034  179590995428478_422375854483323                  0               1   
2035  125472670805257_525103854175468                  0               1   
2036  179590995428478_377568608964048                  1               0   
2037  179590995428478_341070505947192                  0               1   
2038    60686173217_10151073988283218                  0               1   

      Feedback_pred  
0                 0  
1                 0  
2                 0  